<img src="imgs/header.png" width="100%">

---------------

# Unit III: Probabilistic filtering
#### Inferring user intention in a noisy world
<b>[John Williamson](http://johnhw.com)</b> 

----

    All theorems are true. 
    All models are wrong. 
    And all data are inaccurate. 

    What are we to do? 
    We must be sure to remain uncertain.

-- *[Leonard A. Smith, Proc. International School of Physics ``Enrico Fermi", (1997)](http://www2.maths.ox.ac.uk/~lenny/fermi96_main_abs.html)* 

# Introduction 

-----------------
### What are we going to do?
We will:
* show how to represent interaction problems as inference;
* discuss how probabilistic filters can be used to attack these inference problems;
* specifically show how motion-based interfaces can use probabilistic filtering to increase robustness.

**The main purpose of this tutorial is to think about interaction problems as inference, and introduce some practical approaches to solve them.**

### What will we *practically* do?
* We will build a 2D mouse gesture recognizer using a hybrid discrete/continuous particle filter. This will be a simple, robust classifier with rich feedback opportunities.
<img  src="imgs/Capture.PNG" width="800px"/>

* We will build an algorithm to track finger pose from raw capacitive touch sensor readings using a continuous particle filter. We will find this works well even with a poorly specified model.

------

#### What is probabilistic filtering ?
One view on interaction is to see user intentions as **unknown values** which are partially observed through input sensors. The time series of inputs from the user only give a partial, noisy, incomplete view of intention inside the user's head. 

Probabilistic filtering **(PF)** tracks the evolution of some unknown variables *[user intentions]* given observed evidence *[user input]*, in a way that is **robust**. Probabilistic filters infer a **distribution** over possible hidden (unobserved) variables, updating them over time. They are inherently **uncertain**, as they represent degrees of belief, and **dynamic**, as they explicitly model changing state over time.

<img src="imgs/brain_inference.png">

#### Simulation viewpoint
These filters are really *simulators*. They *simulate* how possible user behaviours might unfold over time. In some probabilistic filters, hundreds of parallel simulators are run, each with slightly different parameters. In all cases, the simulations are adjusted online to better match observed reality. The internal parameters that drive the simulation are the *unknown variables* we want to infer and the *evidence* is the observed reality that adjusts the simulation parameters.

#### Properties
Probabilistic filtering is:

| Property | Why  |
|----------|------|
|**Bayesian**  |  Represents degrees of belief using probability distributions    |
|**predictive**  |  Works by comparing predictions with reality   |
|**generative** |  Involves generating (i.e. simulating) behavior   |

-----
Probabilistic filtering is an **inverse probability** approach, and it requires that we think of interaction from an unique perspective. We have to explicitly be able to write down:

* what we want to know (i.e. the **state space of intention**);
* how that will change over time (i.e. the **dynamics of intention**);
*  a model that *if we knew what the user intention was, what the expected behavior would be* (i.e. a **generative function mapping intention -> expected user inputs**).

Note that this last point is the **inverse** of the typical way of approaching this problem, where we would try and find a mapping from a sensors to intention, by design or by learning. 

### Why is this computational HCI?
Probabilistic filtering means writing down an **executable, statistical model** of user behavior, then **running an inference algorithm** that updates beliefs based on the way observations evolve. The **parameters** of the filter can be **learned from data**.

This has four key elements of computational interaction:
* an explicit mathematical model of user-system behavior;
* a way of updating that model with observed data from users;
* an algorithmic element that, using this model, can apply computational power to improving interaction;
* the ability to simulate or synthesize elements of the expected user-system behavior.

It satisfies the requirement that better interfaces can be achieved via:
* improved modeling;
* better data collection;
* more powerful algorithms;  
* or increased computational power, 

rather than the workhorses of traditional HCI:
* more design ingenuity;
* and stronger evaluation.


### What are competitive approaches?
#### **Crafted mappings**
**where we try to find (by hand) transforms from sensors to intentions that are  simple or obvious.**

**Example:** a button, which has two physical states, and maps on to two intentional states via two electrical states. Pushed down = current flows = user intended to switch on. The mapping from electrical states to intentional states is **designed.**
<img src="imgs/undo.jpg">
*[Image credit: David Singleton via flickr.com CC-BY 2.0]*

#### **Machine learned mappings**
**where we train a system to recognize a class of input patterns as being representative of an intended behavior. **
**Example:** Finger gesture recognizer; hundreds of examples of many users performing one of N multi-touch gestures are recorded. These are used to train a random forest to classify the intended gesture. The mapping from electrical states (capacitive sensors) to intentional states is **learned**.

<img src="imgs/svm.jpg" width="300px">
*[Image credit: Elisfm - via Wikimedia Commons; public domain]*

### Benefits
* **Robustness to noise** PFs work well even with input sensors that are noisy.
* **Robustness to poorly specified models** PFs can cope predictably even if our models are bad.
* **Robustness to intermittence** PFs can continue to sensibly interpolate when input cuts out.
* **Uncertainty estimates** PFs *know how certain they are* and this can be used in the interaction design.
* **Decoupled from real-time** PFs can infer past (smoothing), present (filtering) and future (forecasting).
* **Easy fusion of multiple input sensors** PFs are often used to solely to fuse together multiple inputs from different sensors.
* **Better feedback** PFs  offer the opportunity to give users rich insight into the process of intention decoding.
* **Flexible modeling** PFs can incorporate both fundamental modeling (e.g. physiological or cognitive models) and data-driven machine learning.

### History
* 1960s Kalman filter (Swerling, Kalman, Bucy), Extended Kalman Filter (Schmidt)
* late 1960-1990s Particle filter / sequential Monte Carlo
* 1992 Bootstrap filter (Gordon)
* 1995 Unscented Kalman Filter (Uhlmann)
* 1998 Condensation: particle filter for vision problems (Isard and Blake) 

**We will base our model in this unit roughly on the algorithm variant proposed by Isard and Blake.**

# Principles 
-------

> Interaction is the process of driving a system into a state compatible with user intentions.

There are many perspectives on interaction from this stance, including:

| Perspective   | Burden | Characteristic                         |
|---------------|--------|----------------------------------------|
| Communication | User   | User gets information into the system. |
| Control       | Split  | User drives state towards intention.   |
| Inference     | System | Systems infers what user intention is. |

### Interaction as inference
If we view interaction as inference of intention, there are three elements:
* **Interaction is inference**; it is the process of inferring a hidden variable: what the user wants a system to do. 
* **Observations are noisy and incomplete** What a system sees is a distorted and incomplete representation of user actions in the world, which are in turn a noisy representation of internal intentions (your hand does not always go where you want it...)
* **Interaction occurs over time** Interaction is a *process* that evolves over time. Information flow is not instantaneous.

<img src="imgs/brainspace.png" width="100%">

### Overview diagram



<img src="imgs/control_loop.png">



Notation:
* We have a sequence of states over time, indexed by $t$
* $X_t$ the variable we want to know (at time $t$). 
* $Y_t$ the variable we can observe.


* We want to compute $P(X_t|Y_t)$ (the **inverse problem**). 
* We use a **forward model** $P(Y_t|X_t)$ to infer this.
* We need to define two functions: ${\bf\hat{y_t}} = f({\bf \hat{x}}_t)$ (the **observation function**) and $\hat{\bf x}_{t} = g(\hat{\bf x}_{t-1})$ (the **dynamics** or **process function**).

* $f$ and $g$ are often very simple functions.

<img src="imgs/stochastic.png" width="75%">

### Use case
### Problem description
We are going to solve xxx

#### Algorithm
We will use the **particle filter** algorithm (technically the **SIR** variant, which is the simplest to understand).

A particle filter requires that we specify:
* A **dynamics function** that predicts how we expect the world to evolve, which takes 
$\hat{\bf{x}}_t \rightarrow \hat{\bf x}_{t+1}$
* An **observation function** that predicts what we expect to observe, given a hypothesized state $\hat{\bf y}_t = \hat{\bf{x}_t}$
* A function that, given a hypothesized observation $\hat{\bf y}_t$, can be used to compute $p(\hat{\bf y}_t|{\bf y}_t)$. This is performed by computing weights $w_i$ for each particle $i$ using a **weight function**, and then normalizing to produce a probability:
$$p^{(i)}(\hat{\bf y}_t|{\bf y}_t) = \frac{w_i}{\sum_j w_j}$$
* A set of **prior distributions** that specify our initial guesses for $\hat{\bf x}_t$

### Basic filtering
We will first implement a basic particle filter that can track a very simple one dimensional time series. 

Then, we will show how this can generalise to an interesting HCI problem.

In [ ]:
# import the things we need
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pykalman, pfilter
import ipywidgets
import IPython
import matplotlib, matplotlib.colors
matplotlib.rcParams['figure.figsize'] = (14.0, 8.0)
%matplotlib inline

### Some test data
To test the particle filter, we will try and track a very simple, 1D sine wave:
$$Y_t=\sin(t)$$

In [ ]:
t = np.linspace(0,20,100).reshape(-1,1)
x = np.sin(t)
plt.plot(t,x)

### Simple model
We will use a very simple model

* **Dynamics**
We assume that there are no predictable dynamics, just some Gaussian noise $X_t = X_{t+1} + \epsilon,\  \epsilon \sim N(0,\sigma_p)$

In [ ]:
### Identity Example
def dynamics(x):
    # tomorrow is the same as today
    # but slightly randomly different
    sigma_p = 0.2
    return x+np.random.normal(0,sigma_p,x.shape)

We can show what this dynamics model looks like, if we had no resampling step in the filter. This makes predictions without ever having seen any data.

In [ ]:
def simulate_dynamics(priors, dynamics, steps, n_runs=1):
    runs = []
    def simulate_run():
        x = np.array([p.rvs() for p in priors])
        xs = [x]    
        for i in range(steps):
            x = dynamics(x)
            xs.append(x)
        return xs    
    return np.array([simulate_run() for j in range(n_runs)])
    
simulated = simulate_dynamics(priors=[norm(0,1)], dynamics=dynamics, steps=200, n_runs=20)   
plt.plot(simulated[:,:,0].T, alpha=0.3, c='C0');
plt.xlabel("Time step")
plt.ylabel("X")
plt.title("Simulated runs")

* **Observation**
We assume that the sensor we measure is the value we want to infer, i.e. $Y_t=X_t$, and $g(X_t)$  is just the identity.

In [ ]:
def observe(x):
    # we observe x directly
    return x[:,0]

* **Weighting**
We weight samples according to how similar they are to the observed output. We use a simple heat kernel:
$$w_i = e^{\left(-\frac{(y-y^\prime)^2}{2\beta^2}\right)}$$
$\beta$ is a parameter that lets us specify how precise our matching between observation and reality is.

Note that this gives more weight to particles that are more similar to the observation: it is a **similarity** function, not a distance function.

In [ ]:
def weight(true_y, hypothesized_y):
    # RBF similarity function
    beta = 0.25
    return np.exp(-np.sum((true_y-hypothesized_y)**2, axis=0)/(0.5*beta**2))

We can see what this function looks like, comparing a true value of 0 with values in [-1, 1], $\beta=0.25$:

In [ ]:
hypo = np.linspace(-1,1,100).reshape(1,100)
plt.plot(hypo.reshape(100,), weight(0, hypo))
plt.axvline(0,c='C1')
plt.xlabel("Hypothesized value")
plt.ylabel("Unnormalised weight")
plt.title("RBF kernel, $\\beta$=0.25")

**Priors**
We assume a very simple prior on $X$; that it is normally distributed, mean 0, variance 1, $X_0 \sim N(0,1)$

In [ ]:
from scipy.stats import norm

# we assume that, before seeing any evidence, that the particles are 
# normally distributed about 0, with std. dev. 1.0
prior = [norm(0,1)] # x ~ N(0,1)

In [ ]:
## Filter creation


In [ ]:
pf_simple = pfilter.ParticleFilter(initial=prior, 
                                    observe_fn=observe,
                                    n_particles=200,                                    
                                    dynamics_fn=dynamics,
                                    weight_fn=weight,                    
                                    resample_proportion=0.02)

In [ ]:
def run_pfilter(pfilter, inputs):
    """Apply a particle filter to a time series of inputs,
    and return the particles, their weights, and the mean
    estimated state"""    
    # reset filter
    pfilter.init_filter()
    particles = []
    weights = []
    means = []    
    # apply to each element of the time series
    for i in range(len(inputs)):    
        pfilter.update([inputs[i]])
        particles.append(pfilter.particles)    
        weights.append(pfilter.weights)
        means.append(pfilter.mean_state)        
    return np.array(particles), np.array(weights), np.array(means)

In [ ]:
def plot_pfilter(time,expected, observed, particles, weights, means):
    """Apply a particle filter to a time series, and plot the
    first component of the predictions alongside the expected
    output."""
    # expected output
    plt.plot(time, expected, 'C1', lw=3)
    plt.plot(time, observed, 'C3', lw=3)
    
    # particles 
    ts = np.tile(time[:,None], particles.shape[1]).ravel()
    weights =  weights.ravel()    
    rgba_colors = np.zeros((len(weights),4))
    rgba_colors[:,0:3] = matplotlib.colors.to_rgb('C2')
    weights *= 10
    rgba_colors[:, 3] = np.clip(weights, 0, 1)
    plt.scatter(ts, particles[:,:,0].ravel(),  c=rgba_colors, s=2)
    # mean estimation
    plt.plot(time, means, 'C0--', lw=2)
    # legend
    plt.legend(["True", "Observed", "Mean estimate", "Particle"])
    plt.xlabel("Time")
    plt.ylabel("X")
    plt.title("Particle filter estimate")
  

In [ ]:
noise = np.random.normal(0,0.002, x.shape)
particles, weights, means = run_pfilter(pf_simple, x+noise)
plot_pfilter(t, x, x, particles, weights, means)
plt.ylim(-1.5, 1.5)

#### Things to note
* This is a trivial model, but still tracks "complex" functions, because it is adapting to observations



## A more interesting example
Imagine we wanted to infer the **phase** of the oscillator driving this sine wave. The phase variable is not observable, but we want to infer it from the observed oscillation. Furthermore, we want the *unwrapped* phase, i.e. we expect the phase to monotonically increase.

We can encode these assumptions in our model, then see if the particle filter is able to infer the hidden parameter over time.

* **Observation**
We postulate an observation model:
$$Y_t = \sin(X_t)$$
i.e. that what we see is the effect of sine on a hidden variable $X_t$.
Because we defined $Y_t=\sin(t)$, we are actually trying to infer $t$.

* **Dynamics**
We assume that we have a very simple dynamical system in discrete time, where we have a velocity and a position.
$$X_t = \begin{bmatrix}x \\ \dot{x}\end{bmatrix},$$ and 
$$X_{t+1} = X_t + \begin{bmatrix}\dot{x} \\ 0 \end{bmatrix}.$$

This means that if we start linearly increasing or decreasing at a certain rate, we should expect to keep doing so.

* **Priors**
We again assume that the initial distribution is normally distributed.


In [ ]:
### Example
def linear_dynamics(x):        
    nx = np.dot(x,np.array([[1,0],
                            [1,1]]))        
    process_sigmas = [0.1, 0.001] # how much noise for x and dx    
    nx += np.random.normal(0,process_sigmas,x.shape)
    return nx

def observe_sin(x):    
    # y = sin(x)    
    return np.sin(x[:,0])

def weight_sin(true_y, hypothesized_y):
    beta = 0.5 # how like the observations we expect our simulations to be
    return np.exp(-np.sum((true_y-hypothesized_y)**2, axis=0)/beta)
    
from scipy.stats import norm
prior = [norm(0,1), norm(0,0.25)] 

pf_sin = pfilter.ParticleFilter(initial=prior, 
                                observe_fn=observe_sin,
                                n_particles=200,
                                dynamics_fn=linear_dynamics,
                                weight_fn=weight_sin,                    
                                resample_proportion=0.01)

In [ ]:
particles, weights, means = run_pfilter(pf_sin, x)
plot_pfilter(t, t, x, particles, weights, means)

#### Things to note
* The particle filter was able to infer the hidden state, despite only having a forward model (i.e knowledge of $\sin(x)$, **not** $\sin^{-1}(x)$)
* It correctly unwrapped phase, because we primed it with the dynamics model to expect values that would increase at a linear rate.
* This problem results in *multimodal* distributions, because there are an infinite number of solutions to $y=sin(x)$ because we can add any multiple of $2\pi$ without changing anything.
We can see these as fainter lines on the particle plot.
* This means that the particle mean is not actually a good estimate in this case! (we could use the median here instead, but this is harder in higher dimensions)

### Key algorithm summary
| Algorithm       | Dynamics       | State distribution | Efficiency | Optimizable |
|-----------------|----------------|--------------------|------------|-----------|
| Particle        | Arbitrary      | Arbitrary          | Low        | Tricky    |
| Kalman          | Linear         | Gaussian           | Very high  | Easy      |
| Extended Kalman | Locally linear | Gaussian           | High       | Yes       |
| Unscented Kalman| Arbitrary      | Gaussian           | High       | Tricky    |
| HMM             | Transitions    | Discrete           | High       | Yes       |

* Dynamics: permissible state transition functions (i.e. how we go from now to the next timestep).
* State distribution: distribution type for representing current state. Gaussian distributions are very efficient, but can't represent multiple modes.
* Efficiency: computational efficiency.
* Optimizable: is there an algorithm to optimize the  parameters of the filter be *automatically* given training data?


### Gallery
Research papers here (thumbnail + link), short description of why cool

### Pitfalls
Hands-on guru knowledge goes here.

# Outlook
---------------------
### Scope and limitations
#### Scope

#### Limitations
* PFs can be computationally intensive to run. 
* Curse-of-dimensionality can make the attractive simplicity of PFs work poorly in practice as the state space expands.
* Sometimes the inverse probability model can be hard to formulate.
* Particle filters are simple and elegant, but inferentially weak.
* Kalman filters are rigid and restrictive, but very inferentially efficient.
* Hybrid approaches (Ensemble Kalman filter, Unscented Kalman Filter, hybrid particle/Kalman filters) can trade these qualities off, but they aren't off the shelf solutions (i.e. you need an expert!).


### Resources
#### Basic
* Read the [Condensation paper](http://vision.stanford.edu/teaching/cs231b_spring1415/papers/isard-blake-98.pdf).
* Read [the Kalman filter in pictures](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/)
* Watch [the particle filter without equations](https://www.youtube.com/watch?v=aUkBa1zMKv4)

#### Advanced
* [A technical but succinct and clear explanation of the particle filter](http://www.cns.nyu.edu/~eorhan/notes/particle-filtering.pdf)
* [A bibliography of particle filter papers](http://www.stats.ox.ac.uk/~doucet/smc_resources.html)

**some more HCI related resources**

------
### Future of probabilistic filtering

#### Learned models

Much use of probabilistic filters has depended on strong mathematical models of the fundamental process. For example, in rocket science, sophisticated physics models were used to specify the Kalman filters used for stable control. 

However, it is becoming increasingly possible to **infer** these models from observations. Techniques such as deep learning (for example variational autoencoders or generative adversarial networks) make it possible to learn very sophisticated *generative models* from observations of
data.  

These models can be dropped into probabilistic filters to produce robust inferential engines for user interaction.